In [2]:
import numpy as np
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
## load data into pandas data_frame
data = pd.read_csv('full_df.csv')
x = data['Left-Diagnostic Keywords'][2][10] + '|,'

In [8]:
# split diagnostic keyword strings into individual labels
data['Left-Diagnostic Keywords'] =  data['Left-Diagnostic Keywords'].str.split(x)
data['Right-Diagnostic Keywords'] = data['Right-Diagnostic Keywords'].str.split(x) 

In [9]:
# get the most common keywords, we will use the 10 most common labels and classify the rest as 'other'
allKeywords = np.concatenate(data['Left-Diagnostic Keywords'])
allKeywords_df = pd.DataFrame(allKeywords)
num_conditions = allKeywords_df[0].value_counts()
num_conditions[0:11]

normal fundus                             2901
moderate non proliferative retinopathy     915
mild nonproliferative retinopathy          523
cataract                                   304
glaucoma                                   282
pathological myopia                        216
dry age-related macular degeneration       212
hypertensive retinopathy                   191
epiretinal membrane                        175
drusen                                     165
macular epiretinal membrane                162
Name: 0, dtype: int64

In [10]:
# associate each class with an orthogonal vector (one-hot encoding)
classes = {'labels':\
           ['normal fundus',\
            'moderate non proliferative retinopathy',\
            'mild nonproliferative retinopathy',
           'glaucoma',\
            'pathological myopia',\
            'dry age-related macular degeneration',\
            'hypertensive retinopathy',\
            'epiretinal membrane',\
            'drusen',\
            'macular epiretinal membrane',
            'other'],\
            'vector': [[1,0,0,0,0,0,0,0,0,0,0],\
                    [0,1,0,0,0,0,0,0,0,0,0],\
                    [0,0,1,0,0,0,0,0,0,0,0],\
                    [0,0,0,1,0,0,0,0,0,0,0],\
                    [0,0,0,0,1,0,0,0,0,0,0],\
                    [0,0,0,0,0,1,0,0,0,0,0],\
                    [0,0,0,0,0,0,1,0,0,0,0],\
                    [0,0,0,0,0,0,0,1,0,0,0],\
                    [0,0,0,0,0,0,0,0,1,0,0],\
                    [0,0,0,0,0,0,0,0,0,1,0],\
                    [0,0,0,0,0,0,0,0,0,0,1]]}
classes_df = pd.DataFrame(classes)


## using a dict might be better than a data frame for this. gonna leave both for now just in case.
classes_dict = {\
            'normal fundus':                           np.asarray([1,0,0,0,0,0,0,0,0,0,0]),\
            'moderate non proliferative retinopathy' : np.asarray([0,1,0,0,0,0,0,0,0,0,0]),\
            'mild nonproliferative retinopathy':       np.asarray([0,0,1,0,0,0,0,0,0,0,0]),\
            'glaucoma' :                               np.asarray([0,0,0,1,0,0,0,0,0,0,0]),\
            'pathological myopia' :                    np.asarray([0,0,0,0,1,0,0,0,0,0,0]),\
            'dry age-related macular degeneration':    np.asarray([0,0,0,0,0,1,0,0,0,0,0]),\
            'hypertensive retinopathy':                np.asarray([0,0,0,0,0,0,1,0,0,0,0]),\
            'epiretinal membrane':                     np.asarray([0,0,0,0,0,0,0,1,0,0,0]),\
            'drusen':                                  np.asarray([0,0,0,0,0,0,0,0,1,0,0]),\
            'macular epiretinal membrane':             np.asarray([0,0,0,0,0,0,0,0,0,1,0]),\
            'other':                                   np.asarray([0,0,0,0,0,0,0,0,0,0,1])}


In [51]:
#TO-DO
#1. replace all non-important labels in dataframe with 'other'? Boolean Mask or loop, list comprehension
# define two empty arrays for the vectors to go into

#insert 6391 zero vectors

data['left_vectors'] = np.nan
data['right_vectors'] = np.nan

labels_array = np.asarray(classes_df['labels'])
for indx, element in enumerate(data['Left-Diagnostic Keywords']):
    for indx2, label in enumerate(element): 
        if label not in labels_array:
            data.iloc[indx][5][indx2] = 'other'
                 
for indx, element in enumerate(data['Right-Diagnostic Keywords']):
    for indx2, label in enumerate(element):
        if label not in labels_array:
            data.iloc[indx][6][indx2] = 'other'      

         
#2. add two new columns to data to put vectorized labels for each eye.
left_keywords = data['Left-Diagnostic Keywords']
right_keywords = data['Right-Diagnostic Keywords']

left_vectors = np.zeros((6392,11))
right_vectors = np.zeros((6392,11))

for indx, element in enumerate(left_keywords):
    for item in element:
        left_vectors[indx,:] += classes_dict[item]

for indx, element in enumerate(right_keywords):
    for item in element:
        right_vectors[indx,:] += classes_dict[item]      


np.ndarray.all(left_vectors, classes_dict['other'])

#take a look at whats going on at 243

TypeError: only integer scalar arrays can be converted to a scalar index

In [12]:
# we need 
# file_names.txt
# label_vectors.txt

for indx,element in enumerate(left_vectors):
    name = 'labels/'+str(indx)+'_left.txt'
    file = open(name,'w')
    #file.write(str(left_vectors[indx,:]))
    np.savetxt(name, element)
    
for indx,element in enumerate(right_vectors):
    name = 'labels/'+str(indx)+'_right.txt'
    file = open(name,'w')
    #file.write(str(left_vectors[indx,:]))
    np.savetxt(name, element)

KeyboardInterrupt: 